In [16]:
import os
from dotenv import load_dotenv

dotenv_path = '.env'
load_dotenv(dotenv_path)

# Print out the environment variables loaded
for key, value in os.environ.items():
    if 'OPENAI' in key:
        print("OpenAI Key founded")
        #print(f"{key}: {value}")

openai_api_key = os.getenv('OPENAI_API_KEY')

#print(f"OpenAI API Key: {openai_api_key}")

OpenAI Key founded


In [9]:
from langchain_openai import OpenAI
from langchain.prompts.prompt import PromptTemplate

In [6]:
my_template = """
Clasifica el sentimiento del siguiente review.
Sentimiento: Es el review positivo, neutro o negativo?

Review: {review}
"""

In [7]:
llm = OpenAI()

In [10]:
my_prompt = PromptTemplate(
    template=my_template,
    input_variables=["review"]
)

In [12]:
preguntaReview1 = my_prompt.format(review="No me gusto el servicio")
print(preguntaReview1)


Clasifica el sentimiento del siguiente review.
Sentimiento: Es el review positivo, neutro o negativo?

Review: No me gusto el servicio



In [14]:
respuestaReview1 = llm.invoke(preguntaReview1)
print(respuestaReview1)

Sentimiento: Negativo


In [15]:
preguntaReview2 = my_prompt.format(review="La verdad que esto fue un buen servicio")
respuestaReview2 = llm.invoke(preguntaReview2)
print(respuestaReview2)


Positivo
